In [4]:
import pandas as pd
import geopandas as gpd
import os
from dash import Dash, html, dcc, Input, Output

from handlers.general_visualization import GeneralVisualization
from handlers.district_visualization import DistrictVisualization
from handlers.metro_visualization import MetroVisualization
from handlers.general_use import add_borders
from handlers.bus_map import BusMap
from handlers.cercanias import CercaniasMap
from handlers.metro_map import MetroMap

In [5]:
data_path = os.path.join(os.path.dirname(os.getcwd()),  'data', 'pancho_clean')
listings = pd.read_csv(os.path.join(data_path, 'listings_merged.csv'))
calendar = pd.read_csv(os.path.join(data_path, 'calendar.csv'))
gdf = gpd.read_file(os.path.join(data_path, 'neighbourhoods.geojson'))
crimes = pd.read_csv(os.path.join(data_path, 'crimes.csv'))
metro = pd.read_csv(os.path.join(data_path, 'metro.csv'))

In [6]:
gv = GeneralVisualization(listings, gdf)
dv = DistrictVisualization(listings, gdf)
mv = MetroVisualization(listings, metro, gdf)

In [7]:
metro_base_path = os.path.join(os.path.dirname(os.getcwd()), 'data', 'zalla', 'metro.csv')
metro_map = MetroMap(metro_base_path)

cercanias_base_path = os.path.join(os.path.dirname(os.getcwd()), 'data', 'zalla', 'cercanias')
cercanias_map = CercaniasMap(cercanias_base_path)


bus_base_path = os.path.join(os.path.dirname(os.getcwd()), 'data', 'zalla', 'bus_interurbano')
bus_map = BusMap(bus_base_path)

In [ ]:
distritos = list(listings['neighbourhood_group'].unique())

app = Dash(__name__, assets_folder='assets')

app.layout = html.Div(className='container', children=[
	html.H1('Datos de AirBnB en Madrid'),
	
	dcc.Tabs(children=[
		dcc.Tab(label='Información general', children=[
			html.Div(id='general-graphs')
		]),

		dcc.Tab(label='Información por distrito', children=[
			html.H4('Selecciona un distrito:'),
			dcc.Dropdown(
				id='distrito-dropdown-cloropleth',
				options=[{'label': distrito, 'value': distrito} for distrito in distritos],
				value='CENTRO'
			),
			html.Div(id='distrito-graph-cloropleth', className='map-container')
		]),

		dcc.Tab(label='Información con metro', children=[
            html.Div(id='metro-graphs')
		]),

		dcc.Tab(label='Medios de transporte', children=[
            html.Div([
                html.H4('Selecciona el medio de transporte:'),
                dcc.Dropdown(
                    id='transport-selector',
                    options=[
                        {'label': 'Metro', 'value': 'metro'},
                        {'label': 'Cercanías', 'value': 'cercanias'},
                        {'label': 'Autobuses Interurbanos', 'value': 'bus'}
                    ],
                    value='metro',  # valor por defecto
                    style={'width': '50%', 'marginBottom': '20px'}
                ),
                html.Div(id='transport-graphs')
            ])
        ])
	])
])

@app.callback(
	Output('transport-graphs', 'children'),
	Input('transport-selector', 'value')
)
def transport_graphs(selected_transport):

	if selected_transport == 'metro':
		metro_map_visualization = metro_map.create_map()
		metro_map_visualization_html = metro_map_visualization._repr_html_()
		return html.Iframe(
			srcDoc=metro_map_visualization_html,
			style={'width': '100%', 'height': '800px', 'border': 'none'}
		)
	
	elif selected_transport == 'cercanias':
		cercanias_map_visualization = cercanias_map.create_map()
		cercanias_map_visualization_html = cercanias_map_visualization._repr_html_()
		return html.Iframe(
            srcDoc=cercanias_map_visualization_html,
            style={'width': '100%', 'height': '800px', 'border': 'none'}
        )
	
	elif selected_transport == 'bus':
		bus_map_visualization = bus_map.create_map()
		bus_map_visualization_html = bus_map_visualization._repr_html_()
		return html.Iframe(
			srcDoc=bus_map_visualization_html,
			style={
				"width": "100%",
				"height": "600px",
				"border": "none"
			}
		)

	return html.Div("Por favor, selecciona un medio de transporte")

@app.callback(
	Output('metro-graphs', 'children'),
	Input('metro-graphs', 'id')
)
def metro_graphs(_):
	metro_map_borders = mv.get_metro_map(cloropleth=False)
	metro_map_borders_html = metro_map_borders._repr_html_()

	metro_map_cloro = mv.get_metro_map(cloropleth=True)
	metro_map_cloro_html = metro_map_cloro._repr_html_()

	return html.Div([
		html.Iframe(
			srcDoc=metro_map_borders_html,
			style={
				"width": "100%",
				"height": "600px",
				"border": "none"
			}
		),
		html.Iframe(
			srcDoc=metro_map_cloro_html,
			style={
				"width": "100%",
				"height": "600px",
				"border": "none"
			}
		)
	])

@app.callback(
	Output('distrito-graph-cloropleth', 'children'),
	Input('distrito-dropdown-cloropleth', 'value')
)
def district_graphs(distrito):
	district_info = dv.get_district_info(distrito)
	district_cloropleth = dv.get_district_cloropleth(distrito)
	heatmap = dv.get_district_heatmap(distrito)

	choropleth_html = district_cloropleth._repr_html_()
	heatmap_html = heatmap._repr_html_()
	

	return html.Div([
		district_info,
		html.Iframe(
			srcDoc=choropleth_html,
			style={
				"width": "100%",
				"height": "600px",
				"border": "none"
			}
		),
		html.Iframe(
			srcDoc=heatmap_html,
			style={
				"width": "100%",
				"height": "600px",
				"border": "none"
			}
		)
	])


@app.callback(
	Output('general-graphs', 'children'),
	Input('general-graphs', 'id')
)
def general_graphs(_):
	graphs = gv.get_all_graphs()
	cloropleth_map = gv.get_madrid_cloropleth()
	heatmap = gv.get_madrid_heatmap()

	choropleth_html = cloropleth_map._repr_html_()
	heatmap_html = heatmap._repr_html_()
	
	return html.Div([
		graphs,
		html.Iframe(
			srcDoc=choropleth_html,
			style={
				"width": "100%",
				"height": "600px",
				"border": "none"
			}
		),
		html.Iframe(
			srcDoc=heatmap_html,
			style={
				"width": "100%",
				"height": "600px",
				"border": "none"
			}
		)
	])


if __name__ == '__main__':
	app.run(debug=True)